<a href="https://colab.research.google.com/github/InowaR/colab/blob/main/polinomial_matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

# Исходные данные
X = np.array([
    [5, 2, 1, 0], [6, 3, 2, 1], [4, 1, 1, 0],
    [7, 4, 6, 3], [8, 5, 5, 4], [6, 3, 4, 2],
    [3, 8, 7, 9], [4, 9, 8, 8], [2, 7, 6, 7],
], dtype=np.float32) / 10.0

y = np.array([0, 0, 0, 1, 1, 1, 1, 0, 1], dtype=np.float32)

# Функция для создания матрицы дизайна (всех термов)
def create_design_matrix(X):
    n_samples = X.shape[0]
    X1, X2, X3, X4 = X[:, 0], X[:, 1], X[:, 2], X[:, 3]

    # Создаем матрицу дизайна с полиномиальными термами 2-й степени
    design_matrix = np.column_stack([
        np.ones(n_samples, dtype=np.float32),  # bias term
        X1, X2, X3, X4,                       # линейные термы
        X1**2, X2**2, X3**2, X4**2,           # квадратичные термы
        X1*X2, X1*X3, X1*X4,                  # взаимодействия x1
        X2*X3, X2*X4, X3*X4                   # взаимодействия x2, x3, x4
    ])
    return design_matrix

# Создаем матрицу дизайна один раз
design_matrix = create_design_matrix(X)

# Инициализация весов
weights = np.random.uniform(-0.1, 0.1, size=design_matrix.shape[1]).astype(np.float32)

# Параметры обучения
learning_rate = 0.1
n_iterations = 10000

# Обучение с матричными операциями
for _ in range(n_iterations):
    # Вычисляем предсказания для всех образцов сразу
    predictions = np.dot(design_matrix, weights)

    # Вычисляем ошибку для всех образцов
    errors = predictions - y

    # Обновляем веса (матричная операция)
    gradient = np.dot(design_matrix.T, errors) / len(y)
    weights -= learning_rate * gradient

print("Результаты:")
predictions = np.dot(design_matrix, weights)
for i in range(len(X)):
    print(f"{X[i] * 10} -> {predictions[i]:.4f} (Истинное: {y[i]})")

# Можно также вывести среднеквадратичную ошибку
mse = np.mean((predictions - y)**2)
print(f"\nСреднеквадратичная ошибка: {mse:.6f}")

Результаты:
[5. 2. 1. 0.] -> -0.1200 (Истинное: 0.0)
[6. 3. 2. 1.] -> 0.1785 (Истинное: 0.0)
[4. 1. 1. 0.] -> 0.0509 (Истинное: 0.0)
[7. 4. 6. 3.] -> 1.0982 (Истинное: 1.0)
[8. 5. 5. 4.] -> 0.9481 (Истинное: 1.0)
[6. 3. 4. 2.] -> 0.8447 (Истинное: 1.0)
[3. 8. 7. 9.] -> 1.0276 (Истинное: 1.0)
[4. 9. 8. 8.] -> -0.0154 (Истинное: 0.0)
[2. 7. 6. 7.] -> 0.9870 (Истинное: 1.0)

Среднеквадратичная ошибка: 0.009611


In [6]:
import numpy as np

# Исходные данные
X = np.array([
    [5, 2, 1, 0], [6, 3, 2, 1], [4, 1, 1, 0],
    [7, 4, 6, 3], [8, 5, 5, 4], [6, 3, 4, 2],
    [3, 8, 7, 9], [4, 9, 8, 8], [2, 7, 6, 7],
], dtype=np.float32) / 10.0

y = np.array([0, 0, 0, 1, 1, 1, 1, 0, 1], dtype=np.float32)

# Нормализация (Z-score)
X_normalized = (X - X.mean(axis=0)) / (X.std(axis=0) + 1e-8)

# Функция для создания матрицы дизайна
def create_design_matrix(X):
    X1, X2, X3, X4 = X[:, 0], X[:, 1], X[:, 2], X[:, 3]
    return np.column_stack([
        np.ones(len(X), dtype=np.float32),
        X1, X2, X3, X4,
        X1**2, X2**2, X3**2, X4**2,
        X1*X2, X1*X3, X1*X4,
        X2*X3, X2*X4, X3*X4
    ])

design_matrix = create_design_matrix(X_normalized)

# Инициализация весов (Xavier/Glorot)
limit = np.sqrt(6 / (design_matrix.shape[1] + 1))
weights = np.random.uniform(-limit, limit, size=design_matrix.shape[1]).astype(np.float32)

# Параметры обучения
learning_rate = 0.1
lambda_reg = 0.001  # L2-регуляризация
n_iterations = 10000
best_mse = float('inf')
patience = 100
no_improvement = 0

# Обучение с ранней остановкой
for epoch in range(n_iterations):
    predictions = np.dot(design_matrix, weights)
    errors = predictions - y
    gradient = (np.dot(design_matrix.T, errors) + lambda_reg * weights) / len(y)
    weights -= learning_rate * gradient

    mse = np.mean(errors**2)
    if mse < best_mse:
        best_mse = mse
        no_improvement = 0
    else:
        no_improvement += 1
        if no_improvement >= patience:
            print(f"Ранняя остановка на итерации {epoch}")
            break

# Вывод результатов
print("Результаты:")
results = np.column_stack([X * 10, predictions, y])
print("Исходные признаки -> Предсказание (Истинное)")
print(results)
print(f"\nСреднеквадратичная ошибка: {best_mse:.6f}")

Результаты:
Исходные признаки -> Предсказание (Истинное)
[[ 5.0000000e+00  2.0000000e+00  1.0000000e+00  0.0000000e+00
  -4.2898297e-02  0.0000000e+00]
 [ 6.0000000e+00  3.0000000e+00  2.0000000e+00  1.0000000e+00
   4.4485033e-02  0.0000000e+00]
 [ 4.0000000e+00  1.0000000e+00  1.0000000e+00  0.0000000e+00
   1.9938886e-02  0.0000000e+00]
 [ 7.0000000e+00  4.0000000e+00  6.0000000e+00  3.0000000e+00
   1.0128490e+00  1.0000000e+00]
 [ 8.0000000e+00  5.0000000e+00  5.0000000e+00  4.0000000e+00
   9.9161398e-01  1.0000000e+00]
 [ 6.0000000e+00  3.0000000e+00  4.0000000e+00  2.0000000e+00
   9.7317100e-01  1.0000000e+00]
 [ 3.0000000e+00  8.0000000e+00  7.0000000e+00  9.0000000e+00
   1.0043318e+00  1.0000000e+00]
 [ 4.0000000e+00  9.0000000e+00  8.0000000e+00  8.0000000e+00
  -1.8105507e-03  0.0000000e+00]
 [ 2.0000000e+00  7.0000000e+00  6.0000000e+00  7.0000000e+00
   9.9669015e-01  1.0000000e+00]]

Среднеквадратичная ошибка: 0.000578
